<a href="https://colab.research.google.com/github/cynthianekesa/Climate-Change-ChatBot/blob/main/ClimateChangeChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing needed packages

!pip install sentence-transformers
!pip install accelerate -U
!pip install nltk
!pip install datasets
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
#importing libraries

from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import torch.nn.functional as F
import re

from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import json, os

In [3]:
#Loading dataset

from datasets import load_dataset

data = load_dataset("Vinom/ClimateChangeQA")

data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


qa_list.json:   0%|          | 0.00/842k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1277 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question'],
        num_rows: 1277
    })
})

In [4]:
#dataset splitting
train_test_valid = data['train'].train_test_split(test_size=0.2, seed=42)
test_valid = train_test_valid['test'].train_test_split(test_size=0.5, seed=42)

train_ds = train_test_valid['train']
val_ds = test_valid['train']
test_ds = test_valid['test']

print(train_ds)
print(val_ds)
print(test_ds)

Dataset({
    features: ['answer', 'question'],
    num_rows: 1021
})
Dataset({
    features: ['answer', 'question'],
    num_rows: 128
})
Dataset({
    features: ['answer', 'question'],
    num_rows: 128
})


In [5]:
# extracting unique answers from the dataset and assigning labels to each
unique_labels = list(set(train_ds['answer'] + val_ds['answer'] + test_ds['answer']))

# label mapping
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

# saving the label mapping to a json file
label_mapping_path = "./label_mapping.json"
with open(label_mapping_path, 'w') as file:
    json.dump(label_mapping, file)

In [6]:
# Loading a pre-trained sentence transformer model
sent_transf = SentenceTransformer('all-MiniLM-L6-v2')

# Encoding all possible answers
response = sent_transf.encode(list(label_mapping.keys()))

# Saving the answer embeddings
response_path = "./answer_embeddings.npy"
np.save(response_path, response)

# Saving the semantic model
sent_transf_path = "./sent_transf"
os.makedirs(sent_transf_path, exist_ok=True)
sent_transf.save(sent_transf_path)

# Semantic search for question and answer matching
def semantic_search(query, top_k=5):
    # Encoding
    query_embedding = sent_transf.encode([query])

    # Cosine similarity
    similarities = cosine_similarity(query_embedding, response)[0]

    # Geting top-k similar answers
    top_indices = similarities.argsort()[-top_k:][::-1]

    return [(list(label_mapping.keys())[i], similarities[i]) for i in top_indices]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# mapping the text labels to numerical labels
def map_label(example):
    example['label'] = label_mapping[example['answer']]
    return example

# applying the label mappings
train_ds = train_ds.map(map_label)
val_ds = val_ds.map(map_label)
test_ds = test_ds.map(map_label)

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# text cleaning and normalization
def clean_text(text):
    # Lowercase as this removes case sensitivity thereby helps to reduce vocabulary size
    text = text.lower()
    # removing special characters so that we can focus on the meaningful words
    text = re.sub(r'[^a-z\s]', '', text)
    # removing extra whitespace as this ensures a consistent format
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [10]:
# Tokenizing

def tokenize(examples):
    # cleaning the question
    cleaning = [clean_text(question) for question in examples['question']]

    # The BERT tokenizer is used to split the text into tokens, pad the sequences to the same length, and truncate if necessary.
    tokenized_inputs = tokenizer(cleaning, truncation=True, padding="max_length", max_length=512)

    # Adding the label to the tokenized input
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs


# tokenize datasets
tokenized_train = train_ds.map(tokenize, batched=True)
tokenized_val = val_ds.map(tokenize, batched=True)
tokenized_test = test_ds.map(tokenize, batched=True)

# formatting for pytorch
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [11]:
#defining the model
num_labels = len(label_mapping)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# defining training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
 # defining a custom trainer to calculate loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Get labels from inputs dictionary
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels should not be None")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits = logits.contiguous()
        labels = labels.contiguous()

        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

# initializing the trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [14]:
# training the model

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,7.177944
2,No log,7.434958
3,No log,7.734031


TrainOutput(global_step=384, training_loss=7.150657653808594, metrics={'train_runtime': 325.556, 'train_samples_per_second': 9.409, 'train_steps_per_second': 1.18, 'total_flos': 815055382112256.0, 'train_loss': 7.150657653808594, 'epoch': 3.0})

In [15]:
# saving the model
model_path = "./save_model"
os.makedirs(model_path, exist_ok=True)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./save_model/tokenizer_config.json',
 './save_model/special_tokens_map.json',
 './save_model/vocab.txt',
 './save_model/added_tokens.json',
 './save_model/tokenizer.json')

In [16]:
# Evaluating the model
evaluation = trainer.evaluate()
evaluation

{'eval_loss': 7.177943706512451,
 'eval_runtime': 3.5921,
 'eval_samples_per_second': 35.634,
 'eval_steps_per_second': 4.454,
 'epoch': 3.0}

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluating model on test data
predictions = trainer.predict(tokenized_test)

# Geting the unique labels in the predictions
unique_labels_in_data = np.unique(predictions.label_ids)

# Checking if all unique labels are present in the label mapping
filtered_label_mapping = {
    label: idx for label, idx in label_mapping.items() if idx in unique_labels_in_data
}

# Ensuring target_names includes all labels in unique_labels_in_data as strings
# Sorting to maintain the order between labels and target_names
target_names = [
    str(label) for idx, label in sorted(filtered_label_mapping.items(), key=lambda item: item[1])
]

# Function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Ensuring the report uses the correct label names
    report = classification_report(labels, preds, labels=unique_labels_in_data, target_names=target_names)
    conf_matrix = confusion_matrix(labels, preds, labels=unique_labels_in_data)
    return report, conf_matrix

# Computing the metrics
report, conf_matrix = compute_metrics(predictions)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)




Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       1.0
          28       0.00      0.00      0.00       1.0
          37       0.00      0.00      0.00       1.0
          42       0.00      0.00      0.00       1.0
          60       0.00      0.00      0.00       1.0
          85       0.00      0.00      0.00       1.0
          92       0.00      0.00      0.00       1.0
          98       0.00      0.00      0.00       1.0
         105       0.00      0.00      0.00       1.0
         134       0.00      0.00      0.00       1.0
         169       0.00      0.00      0.00       1.0
         172       0.00      0.00      0.00       1.0
         175       0.00      0.00      0.00       1.0
         177       0.00      0.00      0.00       1.0
         183       0.00      0.00      0.00       1.0
         188       0.00      0.00      0.00       1.0
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [18]:
# Generating a response category
# predicts the response category for a given question
def predict_category(question):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Cleaning the question
    cleaned_question = clean_text(question)

    # Getting the BERT prediction
    inputs = tokenizer(
        f"{cleaned_question}",
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_id = torch.argmax(logits, dim=-1).item()
    bert_prediction = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_id)]

    # Getting the semantic search results
    semantic_results = semantic_search(f"{cleaned_question}")

    if bert_prediction in [result[0] for result in semantic_results]:
        return bert_prediction
    else:
        return semantic_results[0][0]



# Example usage:
sample_question = "What are the effects of climate change?"
predicted_answer = predict_category(sample_question)
print(f"Question: {sample_question}")
print(f"Predicted Answer: {predicted_answer}")

Question: What are the effects of climate change?
Predicted Answer: Inaction on climate change could lead to irreversible damage to ecosystems, loss of biodiversity, increased frequency and intensity of natural disasters, displacement of communities, and economic instability. The long-term consequences may include food and water scarcity, health crises, social unrest, and challenges in adapting to a rapidly changing climate.


In [19]:
# Getting model and tokenizer
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [20]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=810f41150b4420d888fe3f5b6adba08a75917420af43c4834a80fe7fac6faec3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [21]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Loading GPT-2 model and tokenizer for Perplexity calculation
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Ensuring models are on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2_model.to(device)  # Move GPT-2 model to device

def calculate_perplexity(text):
    input_ids = gpt2_tokenizer.encode(text, return_tensors='pt').to(device)  # Move input to device
    with torch.no_grad():
        outputs = gpt2_model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss).item()
    return perplexity

# Function to calculate BLEU, ROUGE, and Perplexity scores
def calculate_metrics(prediction, true_output):
    bleu_score = sentence_bleu([true_output.split()], prediction.split())
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge_scorer_obj.score(true_output, prediction)
    perplexity = calculate_perplexity(prediction)
    metrics = {
        "BLEU": bleu_score,
        "ROUGE-1": rouge_scores['rouge1'].fmeasure,
        "ROUGE-L": rouge_scores['rougeL'].fmeasure,
        "Perplexity": perplexity
    }
    return metrics

# Function to generate response category and calculate metrics
def predict_category(question, true_output):
    # Cleaning the question text
    cleaned_question = clean_text(question)

    # Getting BERT prediction
    inputs = tokenizer(
        f"{cleaned_question}",
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(device)  # Move inputs to device

    model.to(device)  # Move BERT model to device

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    bert_prediction = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_class_id)]

    # Getting semantic search results
    semantic_results = semantic_search(f"{cleaned_question}")

    # If BERT prediction is in top semantic results, return it; otherwise, return top semantic result
    if bert_prediction in [result[0] for result in semantic_results]:
        prediction = bert_prediction
    else:
        prediction = semantic_results[0][0]  # Return the top semantic search result

    # Calculating metrics
    metrics = calculate_metrics(prediction, true_output)

    return prediction, metrics

# Testing the model with a sample input
sample_question = "What role does deforestation play in exacerbating climate change?"
true_output = "Deforestation contributes to climate change by reducing the number of trees that can absorb and store carbon dioxide. The removal of trees releases stored carbon back into the atmosphere, leading to increased greenhouse gas concentrations and global warming."

predicted_label, metrics = predict_category(sample_question, true_output)
print(f"Question: {sample_question}")
print(f"True Output: {true_output}")
print(f"Predicted Answer: {predicted_label}")
print(f"Metrics: {metrics}")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Question: What role does deforestation play in exacerbating climate change?
True Output: Deforestation contributes to climate change by reducing the number of trees that can absorb and store carbon dioxide. The removal of trees releases stored carbon back into the atmosphere, leading to increased greenhouse gas concentrations and global warming.
Predicted Answer: Deforestation contributes to climate change by reducing the number of trees that can absorb carbon dioxide from the atmosphere, leading to increased greenhouse gas emissions. The long-term implications for global ecosystems include loss of biodiversity, disruption of water cycles, and exacerbation of climate change effects.
Metrics: {'BLEU': 0.43323251938957036, 'ROUGE-1': 0.6428571428571428, 'ROUGE-L': 0.5714285714285715, 'Perplexity': 10.075514793395996}


In [22]:
# Defining a set of hyperparameters for improving performance

from transformers import TrainingArguments, Trainer

# Defining a list of hyperparameter combinations
hyperparameter_sets = [
    {"learning_rate": 2e-5, "batch_size": 16, "weight_decay": 0.01},
    {"learning_rate": 5e-6, "batch_size": 8, "weight_decay": 0.02},
    {"learning_rate": 1e-6, "batch_size": 32, "weight_decay": 0.005},
    {"learning_rate": 3e-5, "batch_size": 16, "weight_decay": 0.1},
    {"learning_rate": 1e-5, "batch_size": 16, "weight_decay": 0.001},
    {"learning_rate": 1e-6, "batch_size": 8, "weight_decay": 0.01},
    {"learning_rate": 1e-5, "batch_size": 16, "weight_decay": 0.02},
]

best_acc = 0
best_params = None

# Defining a custom Trainer class outside of the loop
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels should not be None")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Make sure logits and labels are contiguous for cross entropy
        logits = logits.contiguous()
        labels = labels.contiguous()

        loss = F.cross_entropy(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Iterating over different hyperparameters
for params in hyperparameter_sets:
    print(f"Training with: {params}")

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=params["batch_size"],
        per_device_eval_batch_size=params["batch_size"],
        warmup_steps=500,
        weight_decay=params["weight_decay"],
        logging_dir="./logs",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        learning_rate=params["learning_rate"],
        report_to="none"
    )

    # Initializing the custom trainer for each set of hyperparameters
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
    )

    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    accuracy = eval_result.get("eval_loss", None)

    print(f"Accuracy for {params}: {accuracy}")

    # Keeping track of best parameters
    if accuracy is not None and accuracy > best_acc:
        best_acc = accuracy
        best_params = params

# Print the best hyperparameters
print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Accuracy: {best_acc}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training with: {'learning_rate': 2e-05, 'batch_size': 16, 'weight_decay': 0.01}


Epoch,Training Loss,Validation Loss
1,No log,7.182325
2,No log,7.205226
3,No log,7.236921


Accuracy for {'learning_rate': 2e-05, 'batch_size': 16, 'weight_decay': 0.01}: 7.18232536315918
Training with: {'learning_rate': 5e-06, 'batch_size': 8, 'weight_decay': 0.02}


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss
1,No log,7.209107
2,No log,7.216673
3,No log,7.224896


Accuracy for {'learning_rate': 1e-06, 'batch_size': 8, 'weight_decay': 0.01}: 7.209107398986816
Training with: {'learning_rate': 1e-05, 'batch_size': 16, 'weight_decay': 0.02}


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,7.215486
2,No log,7.225924
3,No log,7.244209


Accuracy for {'learning_rate': 1e-05, 'batch_size': 16, 'weight_decay': 0.02}: 7.2154860496521

Best Hyperparameters:
{'learning_rate': 1e-05, 'batch_size': 16, 'weight_decay': 0.02}
Best Accuracy: 7.2154860496521
